In [2]:
import tensorflow as tf
import tensorflow_addons as tfa
import os
import cv2
import math
import xml.etree.ElementTree as ET
import random
random.seed(10)
import numpy as np

In [3]:
input_dir = "./JPGimages"
annotations_dir = "./Annotations"

#listing all the images and their annotations
img_files = os.listdir(input_dir)
img_files.remove('desktop.ini')
annotation_files = os.listdir(annotations_dir)
print(img_files)
print(annotation_files)

FileNotFoundError: [WinError 3] The system cannot find the path specified: './JPGimages'

In [36]:
# random hue and saturation

#iterate over the input directory
for image_name in os.listdir(input_dir):
    if image_name.endswith(".jpg"):
        # Load the image and its annotation
        image_path = os.path.join(input_dir, image_name)
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=3)

        # Perform random hue and saturation augmentation
        image = tf.image.random_hue(image, 0.08)
        image = tf.image.random_saturation(image, 0.6, 1.6)

        # Save the augmented image
        output_image_path = os.path.join(input_dir, image_name.replace('.jpg',' randomHS.jpg'))
        tf.io.write_file(output_image_path, tf.image.encode_jpeg(image).numpy())

        # copy the annotation
        annotation_path = os.path.join(annotations_dir, image_name.replace(".jpg",".xml"))
        output_annotation_path = os.path.join(annotations_dir, image_name.replace(".jpg"," randomHS.xml"))
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        tree.write(output_annotation_path)

In [68]:
# rotate clockwise by multiple of 90 degrees
for image_name in os.listdir(input_dir):
    if image_name.endswith(".jpg"):
        # Load the image and its annotation
        image_path = os.path.join(input_dir, image_name)
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=3)

        # Read the annotation file
        annotation_path = os.path.join(annotations_dir, image_name.replace(".jpg",".xml"))
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        
        # Get the size of the image
        size = root.find("size")
        width = int(size.find("width").text)
        height = int(size.find("height").text)
        
        # Perform the rotation on the image
        angle = random.choice([90,180,270])
        factor = int(angle/90)
        image = tf.image.rot90(image, k=factor)
        
        angle = math.radians(angle)

        # Perform the rotation on the bounding boxes
        for obj in root.iter("object"):
            bndbox = obj.find("bndbox")
            xmin = int(bndbox.find("xmin").text)
            ymin = 300 - int(bndbox.find("ymin").text)
            xmax = int(bndbox.find("xmax").text)
            ymax = 300 - int(bndbox.find("ymax").text)
            
            # Find the center of the bounding box
            center_x = 150
            center_y = 150
            
            # Calculate the new bounding box coordinates
            xmin_new = round(center_x + (xmin - center_x) * math.cos(angle) - (ymin - center_y) * math.sin(angle))
            ymin_new = round(center_y + (xmin - center_x) * math.sin(angle) + (ymin - center_y) * math.cos(angle))
            xmax_new = round(center_x + (xmax - center_x) * math.cos(angle) - (ymax - center_y) * math.sin(angle))
            ymax_new = round(center_y + (xmax - center_x) * math.sin(angle) + (ymax - center_y) * math.cos(angle))
            
            ymin_new = 300 - round(ymin_new)
            ymax_new = 300 - round(ymax_new)
            
            xmin1 = min([xmin_new,xmax_new])
            xmax1 = max([xmin_new,xmax_new])
            ymin1 = min([ymin_new,ymax_new])
            ymax1 = max([ymin_new,ymax_new])
        
            # Update the bounding box coordinates in the annotation file
            bndbox.find("xmin").text = str(xmin1)
            bndbox.find("ymin").text = str(ymin1)
            bndbox.find("xmax").text = str(xmax1)
            bndbox.find("ymax").text = str(ymax1)
        
        # Save the augmented image
        output_image_path = os.path.join(input_dir, image_name.replace('.jpg',' randomR.jpg'))
        tf.io.write_file(output_image_path, tf.image.encode_jpeg(image).numpy())

        # copy the annotation
        output_annotation_path = os.path.join(annotations_dir, image_name.replace(".jpg"," randomR.xml"))
        tree.write(output_annotation_path)
        

In [1]:
# rotate image by an angle
for image_name in os.listdir(input_dir):
    if image_name.endswith(".jpg"):
        # Load the image and its annotation
        image_path = os.path.join(input_dir, image_name)
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image, channels=3)
        degree = random.choice(np.arange(10,15,0.4))
        angle = degree * math.pi / 180
        image = tfa.image.rotate(image,angle,"bilinear","nearest")
        print(degree)
        
        # Read the annotation file
        annotation_path = os.path.join(annotations_dir, image_name.replace(".jpg",".xml"))
        tree = ET.parse(annotation_path)
        root = tree.getroot()
        
        # Get the size of the image
        size = root.find("size")
        width = int(size.find("width").text)
        height = int(size.find("height").text)
        

        # Perform the rotation on the bounding boxes
        for obj in root.iter("object"):
            bndbox = obj.find("bndbox")
            xmin = int(bndbox.find("xmin").text)
            ymin = int(bndbox.find("ymin").text)
            xmax = int(bndbox.find("xmax").text)
            ymax = int(bndbox.find("ymax").text)
            
            ymin = 300 - ymin
            ymax = 300 - ymax
            
            # Find the center of the bounding box
            center_x = 150
            center_y = 150
           
            # Calculate the new bounding box coordinates
            xmin_new = round(center_x + (xmin - center_x) * math.cos(angle) - (ymin - center_y) * math.sin(angle))
            ymin_new = round(center_y + (xmin - center_x) * math.sin(angle) + (ymin - center_y) * math.cos(angle))
            xmax_new = round(center_x + (xmax - center_x) * math.cos(angle) - (ymax - center_y) * math.sin(angle))
            ymax_new = round(center_y + (xmax - center_x) * math.sin(angle) + (ymax - center_y) * math.cos(angle))
            
            ymin_new = 300 - ymin_new
            ymax_new = 300 - ymax_new
            
            xmin1 = min([xmin_new,xmax_new])
            xmax1 = max([xmin_new,xmax_new])
            ymin1 = min([ymin_new,ymax_new])
            ymax1 = max([ymin_new,ymax_new])
            
            # calculate factors by which bounding box width might increase            
            width_reduce_factor = 0.8 - 0.02 * (degree-10)
            reduced_width = width_reduce_factor * (xmax1-xmin1)
            center_x = (xmax1 + xmin1)/2
            
            xmin1 = round(center_x - reduced_width/2)
            xmax1 = round(center_x + reduced_width/2)
            
            reduced_height = (ymax-ymin)-(ymax1-ymin1)
            ymin1 = round(ymin1 - reduced_height/2)
            ymax1 = round(ymax1 + reduced_height/2)
            
            # Update the bounding box coordinates in the annotation file
            bndbox.find("xmin").text = str(xmin1)
            bndbox.find("ymin").text = str(ymin1)
            bndbox.find("xmax").text = str(xmax1)
            bndbox.find("ymax").text = str(ymax1)
        
        # Save the augmented image
        output_image_path = os.path.join(input_dir, image_name.replace('.jpg',' randomAR.jpg'))
        tf.io.write_file(output_image_path, tf.image.encode_jpeg(image).numpy())

        # copy the annotation
        output_annotation_path = os.path.join(annotations_dir, image_name.replace(".jpg"," randomAR.xml"))
        tree.write(output_annotation_path)

NameError: name 'os' is not defined

0.7000000000000001
